## Hate Speech Detection Training in SageMaker

## Install requre package

In [2]:
!pip install -qq "sagemaker>=2.48.0" --upgrade
!pip install -qq torch==1.7.1 --upgrade
!pip install -qq sagemaker-huggingface-inference-toolkit 
!pip install -qq transformers==4.6.1 "datasets[s3]"
!pip install -qq ipywidgets
!pip install -qq watermark 
!pip install -qq "seaborn>=0.11.0"

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.23.2 requires botocore==1.25.2, but you have botocore 1.25.9 which is incompatible.
awscli 1.23.2 requires PyYAML<5.5,>=3.10, but you have pyyaml 6.0 which is incompatible.
awscli 1.23.2 requires rsa<4.8,>=3.1.2, but you have rsa 4.8 which is incompatible.
aiobotocore 1.2.2 requires botocore<1.19.53,>=1.19.52, but you have botocore 1.25.9 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sagemaker 2.88.3 requires boto3>=1.20.21, but you have boto3 1.16.43 which is incompatible.
awscli 1.23.2 requires botocore==1.25.2, but you have botocore 1.19.52 which is incompatible.
awscli 1.23.2 requires PyYAML<5.5,>=3.10, but you have pyyaml 6.0 which is incompatible.
aw

In [4]:
!pip install gensim pyvi underthesea tqdm==4.64.0

  Using cached gensim-4.2.0-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (24.1 MB)
  Using cached pyvi-0.1.1-py2.py3-none-any.whl (8.5 MB)
  Using cached underthesea-1.3.4-py3-none-any.whl (7.6 MB)
  Using cached tqdm-4.64.0-py2.py3-none-any.whl (78 kB)
  Using cached smart_open-6.0.0-py3-none-any.whl (58 kB)
  Using cached sklearn_crfsuite-0.3.6-py2.py3-none-any.whl (12 kB)
  Using cached Unidecode-1.3.4-py3-none-any.whl (235 kB)
  Using cached underthesea_core-0.0.4_alpha.10-cp37-cp37m-manylinux2010_x86_64.whl (581 kB)
  Using cached python_crfsuite-0.9.8-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (965 kB)
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.42.1
    Uninstalling tqdm-4.42.1:
      Successfully uninstalled tqdm-4.42.1


In [8]:
import sagemaker
import datasets
import pandas as pd

## 1. Setup Sagemaker

In [9]:
sess = sagemaker.Session()
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::556483467288:role/LabRole
sagemaker bucket: sagemaker-us-east-1-556483467288
sagemaker session region: us-east-1


## 2. Load dataset from S3

In [10]:
bucket='hate-speech-vi'
data_key = 'output_1651908170/part-00000-943eac92-52ff-4360-b862-1747d5c77414-c000.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)

s3_data = pd.read_csv(data_location)

## 3. Data preprocessing and train test split

In [70]:
import pandas as pd
from datasets import Dataset
import re
import gensim # thư viện NLP
from pyvi import ViTokenizer, ViPosTagger # thư viện NLP tiếng Việt
from pandas import DataFrame
from underthesea import word_tokenize

### Load dataset by data wrangling from s3

In [69]:
dataset = Dataset.from_pandas(s3_data)

In [52]:
dataset = dataset.train_test_split(0.2)

In [53]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 55612
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 13903
    })
})

### Filter null value

In [54]:
dataset = dataset.filter(lambda example: example['text'] is not None)

## Vietnamese Preprocessing

In [74]:
def preprocess(row):
    value = row['text']
  # Biến đổi tất cả thành chữ thường
    value = value.lower()
    #Loại bỏ kí tự số
    pattern1 = '\d+'
    valueNew = re.sub(pattern1, '', value)
    #Xóa khoảng trắng đầu và cuối chuỗi
    valueNew = valueNew.strip()
    # Loại bỏ các kí tự đặc biệt
    valueNew = gensim.utils.simple_preprocess(value)
    valueNew = ' '.join(valueNew)
    # Sau khi loại bỏ kí tự đặc biệt thì file sẽ bị mất nhận diện chuỗi -> thêm dấu nháy vào chuỗi
    valueNew = '"' + str(valueNew) + '"'
    #Chuẩn hóa ngôn ngữ tiếng việt sử dụng pyvi
    value = gensim.utils.tokenize(valueNew)

    value = " ".join(list(value))
    valueNew = ViTokenizer.tokenize(value)
    # Loại bỏ thẻ tag html, css, js...
    valueNew = re.sub("(<.*?>)", "", valueNew)

    valueNew = re.sub("(url photos from .*? post)", "", valueNew)
    valueNew = re.sub("updated .*? status", "", value)
    valueNew = word_tokenize(valueNew, format="text")
    
    
    row['text'] = valueNew
    return row

In [55]:
dataset.map(lambda x: preprocess(x))

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 55609
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 13902
    })
})

In [57]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")


def tokenize_function(examples):
#     print(examples)
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## 4. Define Tokenizer and Model

In [58]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42)
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

### Load PhoBert Model from Huggingface Hub

In [59]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base", num_labels=3)

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['

### Define metric

In [60]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

In [78]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

## 5. Train model

In [79]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="hate_vi",
    evaluation_strategy="epoch",
    num_train_epochs=3,
    logging_steps = 100,
    eval_steps = 100)

In [64]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 6131
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2301


Epoch,Training Loss,Validation Loss,Accuracy
1,0.863600,0.816645,0.657000
2,0.686500,0.725395,0.688000
3,0.490600,0.818582,0.691000


Saving model checkpoint to hate_vi/checkpoint-500
Configuration saved in hate_vi/checkpoint-500/config.json
Model weights saved in hate_vi/checkpoint-500/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
Saving model checkpoint to hate_vi/checkpoint-1000
Configuration saved in hate_vi/checkpoint-1000/config.json
Model weights saved in hate_vi/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to hate_vi/checkpoint-1500
Configuration saved in hate_vi/checkpoint-1500/config.json
Model weights saved in hate_vi/checkpoint-1500/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ig

TrainOutput(global_step=2301, training_loss=0.7270974656311987, metrics={'train_runtime': 1788.6965, 'train_samples_per_second': 10.283, 'train_steps_per_second': 1.286, 'total_flos': 2419722546135552.0, 'train_loss': 0.7270974656311987, 'epoch': 3.0})

### 6. Test model performance

In [ ]:
from transformers import TextClassificationPipeline

In [ ]:
print(tokenizer)

PreTrainedTokenizerFast(name_or_path='bert-base-cased', vocab_size=28996, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


In [ ]:
%time
model = model.to('cpu')
tokenizer = tokenizer
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True)
# outputs a list of dicts like [[{'label': 'NEGATIVE', 'score': 0.0001223755971295759},  {'label': 'POSITIVE', 'score': 0.9998776316642761}]]
pipe("bố mẹ mày vô phúc mới đẻ ra mày")

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.54 µs


[[{'label': 'LABEL_0', 'score': 0.006531822960823774},
  {'label': 'LABEL_1', 'score': 0.03879402577877045},
  {'label': 'LABEL_2', 'score': 0.9546741247177124}]]

## 7. Save model and Tokenizer

In [ ]:
trainer.save_model('pho_hate')

Saving model checkpoint to pho_hate
Configuration saved in pho_hate/config.json
Model weights saved in pho_hate/pytorch_model.bin


In [ ]:
tokenizer.save_pretrained('./pho_hate')

tokenizer config file saved in ./pho_hate/tokenizer_config.json
Special tokens file saved in ./pho_hate/special_tokens_map.json
added tokens file saved in ./pho_hate/added_tokens.json


('./pho_hate/tokenizer_config.json',
 './pho_hate/special_tokens_map.json',
 './pho_hate/vocab.txt',
 './pho_hate/bpe.codes',
 './pho_hate/added_tokens.json')

### 8. Push model to Huggingface Hub

In [ ]:
model.push_to_hub('phobert-finetune-hatespeech')